In [1]:
import sys
LLM_UTILS_PATH = "/Users/anuragmishra/Documents/AI Workspace/"
sys.path.insert(0, LLM_UTILS_PATH)

In [2]:
from llm_utils.models_config import openai_llm

In [3]:
import os
from deepagents import create_deep_agent
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools

In [4]:
server_params = StdioServerParameters(
    command="npx",
    # Make sure to update to the full absolute path to your math_server.py file
    args=["@playwright/mcp@latest"],
)

In [5]:
AGENT_PROMPT  = """
"You are an expert web researcher and job finder. You can use the browser tools to navigate websites, fill out forms, and extract information about job listings. 
Your goal is to find relevant job postings based on user queries. Provide concise and accurate results. 

## Instructions:
- Use the tools wisely to gather the required information. Be mindful of login credentials and ensure you handle them securely. 
- Always confirm successful logins before proceeding to search for jobs. If you encounter any issues, report them clearly. Ensure to plan your actions step-by-step before executing them. Keep the username and password in right box
- Check for tools size limitations and avoid exceeding them.
- If the tools size are more than 2000 tokens or contains repeated summaries, condense them clearly.

## Steps to follow:
1. Outline a plan based on the user's request. use 'write_todos' tool to create a step-by-step plan.
2. Execute the plan using the available browser tools.
3. Ensure all steps are executed correctly and efficiently.
4. Return a clear and concise summary of the findings.

Never afraid to update plan if you find new information or obstacles during execution.
"""

In [6]:
from deepagents.backends import FilesystemBackend
from langchain.agents.middleware import ContextEditingMiddleware, ClearToolUsesEdit

In [7]:
agent_task = """Go to likedin.com and Once confirm logged in,
Search for 'Generative AI architect' and find the latest job postings
"""

In [10]:
from langchain.agents.middleware import (
    AgentMiddleware,
    AgentState,
    ModelRequest,
    ModelResponse,
)
from langgraph.runtime import Runtime
from typing import Any, Dict, Optional

In [ ]:
class LoggingMiddleware(AgentMiddleware):
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        for msg in state['messages']:
            # Safely obtain content (works for objects with .content or dict-like messages)
            try:
                content = msg.content
            except Exception:
                try:
                    content = msg.get('content')
                except Exception:
                    content = None

            # If content is a list, truncate string elements and 'text' fields in dict elements
            if isinstance(content, list):
                new_content = []
                for elem in content:
                    if isinstance(elem, str):
                        new_content.append(elem if len(elem) <= 2000 else elem[:2000])
                    elif isinstance(elem, dict) and isinstance(elem.get('text'), str):
                        truncated = dict(elem)
                        if len(truncated['text']) > 2000:
                            truncated['text'] = truncated['text'][:2000]
                        new_content.append(truncated)
                    else:
                        new_content.append(elem)

                # Try to write the truncated list back to the message
                try:
                    msg.content = new_content
                except Exception:
                    try:
                        msg['content'] = new_content
                    except Exception:
                        pass

            print(f"--- Message before model call: {msg}\n")
        print(f"About to call model with {len(state['messages'])} messages")
        return None

    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"Model returned: {state['messages'][-1].content}")
        if state['messages'][-1].tool_calls:
            print(f"--- Full message content: {[tc['name'] for tc in state['messages'][-1].tool_calls]}\n")
        return None


In [20]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        playwright_tools = await load_mcp_tools(session)

        print(f"Loaded tools: {[tool.name for tool in playwright_tools]}")
        # Create and run the agent
        agent = create_deep_agent(
            model=openai_llm,
            tools=playwright_tools,
            system_prompt=AGENT_PROMPT,
            backend=FilesystemBackend(root_dir=".", virtual_mode=True),
            middleware=[LoggingMiddleware(), 
                ContextEditingMiddleware(
                    edits=[
                        ClearToolUsesEdit(
                            trigger=2000,
                            keep=3,
                            clear_tool_inputs=False,
                            exclude_tools=[],
                            placeholder="[cleared]",
                        ),
                    ],
                ),
            ],
        )

        async for chunk in agent.astream({
            "messages": [{"role": "user", "content": agent_task}]
        }, stream_mode="values"):
            # Each chunk contains the full state at that point
            latest_message = chunk["messages"][-1]
            if latest_message.content:
                print(f"Agent: {latest_message.content}")
            elif latest_message.tool_calls:
                print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

Loaded tools: ['browser_close', 'browser_resize', 'browser_console_messages', 'browser_handle_dialog', 'browser_evaluate', 'browser_file_upload', 'browser_fill_form', 'browser_install', 'browser_press_key', 'browser_type', 'browser_navigate', 'browser_navigate_back', 'browser_network_requests', 'browser_run_code', 'browser_take_screenshot', 'browser_snapshot', 'browser_click', 'browser_drag', 'browser_hover', 'browser_select_option', 'browser_tabs', 'browser_wait_for']
Agent: Go to likedin.com and Once confirm logged in,
Search for 'Generative AI architect' and find the latest job postings

Agent: Go to likedin.com and Once confirm logged in,
Search for 'Generative AI architect' and find the latest job postings

--- Message before model call: content="Go to likedin.com and Once confirm logged in,\nSearch for 'Generative AI architect' and find the latest job postings\n" additional_kwargs={} response_metadata={} id='80ff9473-b871-4cf4-a63c-941f8d615df4'

About to call model with 1 messag

CancelledError: 

In [12]:
agent.

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=RunnableCallable(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'model': Node(id='model', name='model', data=model_node(tags=None, recurse=True, explode_args=False, func_accepts={'runtime': ('N/A', <class 'inspect._empty'>)}), metadata=None), 'tools': Node(id='tools', name='tools', data=tools(tags=None, recurse=True, explode_args=False, func_accepts={'config': ('N/A', <class 'inspect._empty'>), 'runtime': ('N/A', <class 'inspect._empty'>)}, _tools_by_name={'write_todos': StructuredTool(name='write_todos', description="Use this tool to create and manage a structured task list for your current work session. This helps you track progress, organize complex tasks, and demonstrate thoroughness to the user.\n\nOnly use this tool if you think it will be helpful in staying organized. If the user's request is trivial and takes less than 3 steps, it is better to NOT use this tool and just do the 

In [16]:
playwright_tools[1]

StructuredTool(name='browser_resize', description='Resize the browser window', args_schema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, metadata={'title': 'Resize browser window', 'readOnlyHint': False, 'destructiveHint': True, 'idempotentHint': None, 'openWorldHint': True}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x116200fe0>)